# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [30]:
import pandas as pd
import requests
import numpy as np

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Contar o número de registros
total_registros = sinasc.shape[0]

# Contar o número de registros não duplicados
registros_nao_duplicados = sinasc.drop_duplicates().shape[0]

# Verificar se há linhas duplicadas
linhas_duplicadas = total_registros != registros_nao_duplicados

# Mostrar os resultados
print("Número total de registros:", total_registros)
print("Número de registros não duplicados:", registros_nao_duplicados)
print("Há linhas duplicadas na base de dados?", linhas_duplicadas)

Número total de registros: 27028
Número de registros não duplicados: 27028
Há linhas duplicadas na base de dados? False


In [20]:

# Contar o número de valores missing por variável
valores_missing = sinasc.isna().sum()

# Visualizar os resultados
valores_missing.to_frame().reset_index()



,index,0
0,ORIGEM,0
1,CODESTAB,115
2,CODMUNNASC,0
3,LOCNASC,0
4,IDADEMAE,0
...,...,...
64,munResUf,0
65,munResLat,1
66,munResLon,1
67,munResAlt,1


In [21]:
# 3) seu código aqui

# Carregar o arquivo CSV apenas com as colunas de interesse
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
                     'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

sinasc_interesse = pd.read_csv('SINASC_RO_2019.csv', usecols=colunas_interesse)

# Contar o número de valores missing por variável
missing_values_interesse = sinasc_interesse.isnull().sum()

print("Número de valores missing por variável nas colunas de interesse:")
print(missing_values_interesse)


Número de valores missing por variável nas colunas de interesse:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [22]:
# 4) seu código aqui

# Remover registros com valores ausentes em 'APGAR5'
sinasc_sem_missing_apgar5 = sinasc_interesse.dropna(subset=['APGAR5'])

# Contar o número de linhas após a remoção de valores ausentes em 'APGAR5'
num_linhas_apgar5_preenchido = sinasc_sem_missing_apgar5.shape[0]

# Contar o número de valores missing nas colunas restantes
missing_values_apgar5_removido = sinasc_sem_missing_apgar5.isnull().sum()

print("Número de linhas após remover registros com APGAR5 não preenchido:", num_linhas_apgar5_preenchido)
print("Número de valores missing por variável nas colunas restantes:")
print(missing_values_apgar5_removido)


Número de linhas após remover registros com APGAR5 não preenchido: 26925
Número de valores missing por variável nas colunas restantes:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [42]:
# 5) seu código aqui
from pprint import pprint

# Carregar o arquivo CSV apenas com as colunas de interesse
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
                     'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

# Substituir o código '9' por NaN em 'ESTCIVMAE' e 'CONSULTAS'
sinasc_interesse['ESTCIVMAE'].fillna(9, inplace=True)
sinasc_interesse['CONSULTAS'].fillna(9, inplace=True)

# Remover registros com valores ausentes em 'APGAR5'
sinasc_sem_missing_apgar5 = sinasc_interesse.dropna(subset=['APGAR5'])

# Contar o número de linhas após a remoção de valores ausentes em 'APGAR5'
num_linhas_apgar5_preenchido = sinasc_sem_missing_apgar5.shape[0]

# Contar o número de valores missing nas colunas restantes
missing_values_apgar5_removido = sinasc_sem_missing_apgar5.isnull().sum()

pprint(sinasc_interesse)


       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos         1.0   
2            1        37        9.0      8 a 11 anos         2.0   
3            1        30        5.0  12 anos ou mais         0.0   
4            1        30        2.0      8 a 11 anos         1.0   
...        ...       ...        ...              ...         ...   
27023        1        32        2.0  12 anos ou mais         1.0   
27024        1        19        1.0      8 a 11 anos         0.0   
27025        1        24        2.0      8 a 11 anos         0.0   
27026        1        21        2.0      8 a 11 anos         1.0   
27027        1        18        5.0      8 a 11 anos         1.0   

              GESTACAO GRAVIDEZ  CONSULTAS  APGAR5  
0      37 a 41 semanas    Única        4.0    10.0  
1      37 a 41 semanas    Única        4.0     9.0  
2      37 a 41 semanas  

In [36]:
# 6) Seu código aqui

# Carregar o arquivo CSV apenas com as colunas de interesse
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
                     'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

# Substituir valores faltantes em 'QTDFILVIVO' por zero
sinasc_interesse['QTDFILVIVO'].fillna(0, inplace=True)

# Substituir o código '9' por NaN em 'ESTCIVMAE' e 'CONSULTAS'
sinasc_interesse['ESTCIVMAE'].fillna(9, inplace=True)
sinasc_interesse['CONSULTAS'].fillna(9, inplace=True)

# Remover registros com valores ausentes em 'APGAR5'
sinasc_sem_missing_apgar5 = sinasc_interesse.dropna(subset=['APGAR5'])

# Contar o número de linhas após a remoção de valores ausentes em 'APGAR5'
num_linhas_apgar5_preenchido = sinasc_sem_missing_apgar5.shape[0]

# Contar o número de valores missing nas colunas restantes
missing_values_apgar5_removido = sinasc_sem_missing_apgar5.isnull().sum()

print("Número de linhas após remover registros com APGAR5 não preenchido:", num_linhas_apgar5_preenchido)
print("Número de valores missing por variável nas colunas restantes:")
print(missing_values_apgar5_removido)



Número de linhas após remover registros com APGAR5 não preenchido: 26925
Número de valores missing por variável nas colunas restantes:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [46]:
# 7) seu código aqui

# Carregar o arquivo CSV apenas com as colunas de interesse
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
                     'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

# Substituir valores faltantes em 'QTDFILVIVO' por zero
sinasc_interesse['QTDFILVIVO'].fillna(0, inplace=True)

# Substituir o código '9' por NaN em 'ESTCIVMAE' e 'CONSULTAS'
sinasc_interesse['ESTCIVMAE'].fillna(9, inplace=True)
sinasc_interesse['CONSULTAS'].fillna(9, inplace=True)

# Remover registros com valores ausentes em 'APGAR5'
sinasc_sem_missing_apgar5 = sinasc_interesse.dropna(subset=['APGAR5'])

# Contar o número de linhas após a remoção de valores ausentes em 'APGAR5'
num_linhas_apgar5_preenchido = sinasc_sem_missing_apgar5.shape[0]

# Preencher 'IDADEMAE' com a mediana
mediana_idade_mae = sinasc_interesse['IDADEMAE'].median()
sinasc_interesse['IDADEMAE'].fillna(mediana_idade_mae, inplace=True)

# Preencher 'ESCMAE' com a moda
moda_esc_mae = sinasc_interesse['ESCMAE'].mode()[0]  # Obtém a moda (o valor mais frequente)
sinasc_interesse['ESCMAE'].fillna(moda_esc_mae, inplace=True)

# Contar o número de valores missing nas colunas restantes
missing_values_apgar5_removido = sinasc_sem_missing_apgar5.isnull().sum()

#Justificativa do porque das decisões:

#IDADEMAE: Preencher com um valor mais provável pode ser uma opção, se houver um valor predominante, como a mediana das idades das mães, por exemplo.

#ESCMAE (escolaridade da mãe): Dependendo da distribuição das escolaridades, podemos optar por preencher com o valor mais comum, ou seja, a moda, caso haja uma escolaridade predominante.

print(sinasc_interesse.head())
print("Primeiras linhas do DataFrame após as mudanças:")
print(sinasc_sem_missing_apgar5.head())



   LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO         GESTACAO  \
0        1        19        5.0      8 a 11 anos         0.0  37 a 41 semanas   
1        1        29        2.0      8 a 11 anos         1.0  37 a 41 semanas   
2        1        37        9.0      8 a 11 anos         2.0  37 a 41 semanas   
3        1        30        5.0  12 anos ou mais         0.0  37 a 41 semanas   
4        1        30        2.0      8 a 11 anos         1.0  37 a 41 semanas   

  GRAVIDEZ  CONSULTAS  APGAR5  
0    Única        4.0    10.0  
1    Única        4.0     9.0  
2    Única        4.0    10.0  
3    Única        3.0    10.0  
4    Única        4.0    10.0  
Primeiras linhas do DataFrame após as mudanças:
   LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO         GESTACAO  \
0        1        19        5.0      8 a 11 anos         0.0  37 a 41 semanas   
1        1        29        2.0      8 a 11 anos         1.0  37 a 41 semanas   
2        1        37        9.

In [47]:
# 8) seu código aqui

# Carregar o arquivo CSV apenas com a coluna 'APGAR5'
sinasc_apgar5 = pd.read_csv('SINASC_RO_2019.csv', usecols=['APGAR5'])

# Definir os intervalos e rótulos para categorização do APGAR5
intervalos = [-1, 3, 5, 7, 10]
rotulos = ['Asfixia Severa', 'Asfixia Moderada', 'Asfixia Leve', 'Normal']

# Categorizar a variável 'APGAR5'
sinasc_apgar5['Categoria_APGAR5'] = pd.cut(sinasc_apgar5['APGAR5'], bins=intervalos, labels=rotulos)

# Contar a frequência de cada categoria
frequencia_categorias = sinasc_apgar5['Categoria_APGAR5'].value_counts()

print("Frequência da categorização do APGAR5:")
print(frequencia_categorias)


Frequência da categorização do APGAR5:
Categoria_APGAR5
Normal              26463
Asfixia Leve          320
Asfixia Severa         74
Asfixia Moderada       68
Name: count, dtype: int64


In [52]:
# 9) seu código aqui

# Renomear as colunas para snake case
sinasc.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

# Visualizar as primeiras linhas do DataFrame após a renomeação
print(sinasc.head())

   origem   codestab  codmunnasc  locnasc  idademae  estcivmae  \
0       1  2679477.0      110001        1        19        5.0   
1       1  2679477.0      110001        1        29        2.0   
2       1  2679477.0      110001        1        37        9.0   
3       1  2516500.0      110001        1        30        5.0   
4       1  2516500.0      110001        1        30        2.0   

            escmae  codocupmae  qtdfilvivo  qtdfilmort  ...  kotelchuck  \
0      8 a 11 anos         NaN         0.0         0.0  ...           5   
1      8 a 11 anos    999992.0         1.0         0.0  ...           5   
2      8 a 11 anos    513205.0         2.0         0.0  ...           5   
3  12 anos ou mais    231205.0         0.0         0.0  ...           4   
4      8 a 11 anos    999992.0         1.0         0.0  ...           5   

  contador munresstatus munrestipo               munresnome  munresuf  \
0        1        ATIVO      MUNIC    Alta Floresta D'Oeste  Rondônia   
1     

In [59]:
import pandas as pd
import numpy as np

# Carregar o arquivo CSV
sinasc = pd.read_csv('SINASC_RO_2019.csv')

# Selecionar colunas de interesse
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
                     'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

sinasc_interesse = sinasc[colunas_interesse]

# Preencher valores faltantes
sinasc_interesse['QTDFILVIVO'].fillna(0, inplace=True)
sinasc_interesse['ESTCIVMAE'].replace(9, pd.NA, inplace=True)
sinasc_interesse['CONSULTAS'].replace(9, pd.NA, inplace=True)

mediana_idade_mae = sinasc_interesse['IDADEMAE'].median()
sinasc_interesse['IDADEMAE'].fillna(mediana_idade_mae, inplace=True)

moda_esc_mae = sinasc_interesse['ESCMAE'].mode()[0]
sinasc_interesse['ESCMAE'].fillna(moda_esc_mae, inplace=True)

# Categorizar o APGAR5
intervalos = [-1, 3, 5, 7, 10]
rotulos = ['Asfixia Severa', 'Asfixia Moderada', 'Asfixia Leve', 'Normal']
sinasc_interesse['Categoria_APGAR5'] = pd.cut(sinasc_interesse['APGAR5'], bins=intervalos, labels=rotulos)

# Renomear as colunas para snake case
sinasc_interesse.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

# Visualizar as primeiras linhas do DataFrame após as mudanças
print("Base de dados após as transformações:")
print(sinasc_interesse.head())

sinasc_interesse.to_excel('SinascNova.xlsx')




C:\Users\gutoc\AppData\Local\Temp\ipykernel_24220\2815096172.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_interesse['QTDFILVIVO'].fillna(0, inplace=True)
C:\Users\gutoc\AppData\Local\Temp\ipykernel_24220\2815096172.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_interesse['ESTCIVMAE'].replace(9, pd.NA, inplace=True)
C:\Users\gutoc\AppData\Local\Temp\ipykernel_24220\2815096172.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

Base de dados após as transformações:
   locnasc  idademae estcivmae           escmae  qtdfilvivo         gestacao  \
0        1        19       5.0      8 a 11 anos         0.0  37 a 41 semanas   
1        1        29       2.0      8 a 11 anos         1.0  37 a 41 semanas   
2        1        37      <NA>      8 a 11 anos         2.0  37 a 41 semanas   
3        1        30       5.0  12 anos ou mais         0.0  37 a 41 semanas   
4        1        30       2.0      8 a 11 anos         1.0  37 a 41 semanas   

  gravidez consultas  apgar5 categoria_apgar5  
0    Única         4    10.0           Normal  
1    Única         4     9.0           Normal  
2    Única         4    10.0           Normal  
3    Única         3    10.0           Normal  
4    Única         4    10.0           Normal  
       LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  \
0            1        19        5.0      8 a 11 anos         0.0   
1            1        29        2.0      8 a 11 anos     